In [1]:
!pip install duckdb
!pip install notebook
!pip install jupysql pandas matplotlib duckdb-engine


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import os
input_path = "./data/geojson/osm21_sports.geojson"
input_filename = os.path.basename(input_path)

In [6]:
import duckdb
from sqlalchemy.engine import create_engine

%load_ext sql
conn = create_engine('duckdb:///data/local.duckdb')
%sql conn --alias duckdb
%config SqlMagic.named_parameters="enabled" 

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
%%sql
INSTALL psql FROM community;
INSTALL spatial;
LOAD spatial;
LOAD psql;


Running query in 'duckdb'

Success


In [11]:
%%sql
CREATE OR REPLACE TABLE point_data AS
SELECT *
FROM ST_Read("{{input_path}}")

Running query in 'duckdb'

Count


In [52]:
import random
import string

conn.raw_connection().remove_function('random_string')

# Define the Python function to generate a random string of length 'n'
def random_string(n: int) -> str:
    return ''.join(random.choices(string.ascii_letters + string.digits, k=n))

# Register the function with DuckDB

conn.raw_connection().create_function('random_string', random_string, side_effects=True)

In [53]:
%sql ROLLBACK

Running query in 'duckdb'

Success


In [54]:
%sql SELECT random_string(10) AS random_string;

Running query in 'duckdb'

random_string
6b2mbcbDPW


In [56]:
%%sql
CREATE OR REPLACE TABLE point_data_centroid AS
WITH centroid AS (
    FROM point_data
    SELECT centroid: geom.ST_Centroid(),
)
FROM centroid
SELECT
    id: ROW_NUMBER() OVER (ORDER BY centroid),
    locationX: centroid.ST_X(),
    locationy: centroid.ST_Y(),
    description: random_string(64)


Running query in 'duckdb'

Count


In [33]:
%sql SELECT * FROM point_data_centroid

Running query in 'duckdb'

id,locationX,locationy,description
1,5.75,59.5166667,OKEzzwBKVsfW86K1ktryjAUP1Xi6WDlhH7RveaxxHSHaAdVXig7lgUhqmnFm7nIB
2,6.75,45.71139,OKEzzwBKVsfW86K1ktryjAUP1Xi6WDlhH7RveaxxHSHaAdVXig7lgUhqmnFm7nIB
3,7.25,61.9719444,OKEzzwBKVsfW86K1ktryjAUP1Xi6WDlhH7RveaxxHSHaAdVXig7lgUhqmnFm7nIB
4,8.5,49.2666667,OKEzzwBKVsfW86K1ktryjAUP1Xi6WDlhH7RveaxxHSHaAdVXig7lgUhqmnFm7nIB
5,11.0,61.4347222,OKEzzwBKVsfW86K1ktryjAUP1Xi6WDlhH7RveaxxHSHaAdVXig7lgUhqmnFm7nIB
6,13.0,66.0,OKEzzwBKVsfW86K1ktryjAUP1Xi6WDlhH7RveaxxHSHaAdVXig7lgUhqmnFm7nIB
7,13.0,66.3513889,OKEzzwBKVsfW86K1ktryjAUP1Xi6WDlhH7RveaxxHSHaAdVXig7lgUhqmnFm7nIB
8,15.5,67.2972222,OKEzzwBKVsfW86K1ktryjAUP1Xi6WDlhH7RveaxxHSHaAdVXig7lgUhqmnFm7nIB
9,15.5,68.7,OKEzzwBKVsfW86K1ktryjAUP1Xi6WDlhH7RveaxxHSHaAdVXig7lgUhqmnFm7nIB
10,16.0,69.2666667,OKEzzwBKVsfW86K1ktryjAUP1Xi6WDlhH7RveaxxHSHaAdVXig7lgUhqmnFm7nIB


In [57]:
%%sql
SELECT COUNT(*) FROM point_data_centroid as data;

Running query in 'duckdb'

count_star()
1982028


In [58]:
%%sql
SUMMARIZE point_data_centroid

Running query in 'duckdb'

column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
id,BIGINT,1,1982028,1831607,991014.5,572162.3440082349,497828,976675,1481278,1982028,0.00
locationX,DOUBLE,-179.9986439,179.87257868463195,2017034,-9.936078170190061,68.12757545869334,-75.16495994519042,2.827785756496136,14.926963358482743,1982028,0.00
locationy,DOUBLE,-77.84672055,78.91086695,2169155,34.536074321214684,24.123391130989003,33.470137129091654,42.54958062168417,49.26518053293296,1982028,0.00
description,VARCHAR,000ALB4g4uZChA7aFWCSgXT5SCF59dRBr4LmiAJdvySmanCBbVxT5pVjq5m8jT1q,zzzytnDBSt89DuUyMobWbEYFI1YPANorL6lAOxnUGyd0jv7DHSsJFdJa1eaD5hcb,2015394,None,None,None,None,None,1982028,0.00


In [59]:
export_filename = input_filename.replace(".geojson", ".csv")
export_path = "./data/csv/" + export_filename

os.makedirs(os.path.dirname(export_path), exist_ok=True)

print(f"Exporting to {export_filename}")
print(f"Exporting to {export_path}")

Exporting to osm21_sports.csv
Exporting to ./data/csv/osm21_sports.csv


In [22]:
%%sql
COPY (SELECT * FROM point_data_centroid) TO '{{export_path}}' (FORMAT CSV, HEADER);

Running query in 'duckdb'

Count
